# Preliminaries
Write requirements to file, anytime you run it, in case you have to go back and recover dependencies.

Latest known such requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp

In [1]:
!pip freeze > kaggle_image_requirements.txt

# Download IMDB Movie Review Dataset
Download IMDB dataset

In [2]:
import random
import pandas as pd

## Read-in the reviews and print some basic descriptions of them

!wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!tar xzf aclImdb_v1.tar.gz

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)


# Define Tokenization, Stop-word and Punctuation Removal Functions
Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters

In [3]:
Nsamp = 1000 # number of samples to generate in each class - 'spam', 'not spam'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token

**Tokenization**

In [4]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        tokens = row.split(" ")[:maxtokens]
    return tokens

**Use regular expressions to remove unnecessary characters**

Next, we define a function to remove punctuation marks and other nonword characters (using regular expressions) from the emails with the help of the ubiquitous python regex library. In the same step, we truncate all tokens to hyperparameter maxtokenlen defined above.

In [5]:
import re

def reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower() # make all characters lower case
            token = re.sub(r'[\W\d]', "", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

**Stop-word removal**

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [6]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')    

# print(stopwords) # see default stopwords
# it may be beneficial to drop negation words from the removal list, as they can change the positive/negative meaning
# of a sentence
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

# Assemble Embedding Vectors

The following functions are used to extract fastText embedding vectors for each review

In [8]:
import time
from gensim.models import FastText, KeyedVectors

!ls

start=time.time()
FastText_embedding = KeyedVectors.load_word2vec_format("../input/jigsaw/wiki.en.vec")
end = time.time()
print("Loading the embedding took %d seconds"%(end-start))

__notebook__.ipynb  aclImdb  aclImdb_v1.tar.gz	kaggle_image_requirements.txt
Loading the embedding took 1183 seconds


In [9]:
def handle_out_of_vocab(embedding,in_txt):
    out = None
    for word in in_txt:
        try:
            tmp = embedding[word]
            tmp = tmp.reshape(1,len(tmp))
            
            if out is None:
                out = tmp
            else:
                out = np.concatenate((out,tmp),axis=0)
        except:
            pass
    
    return out
        

def assemble_embedding_vectors(data):
    out = None
    for item in data:
        tmp = handle_out_of_vocab(FastText_embedding,item)
        if tmp is not None:
            dim = tmp.shape[1]
            if out is not None:
                vec = np.mean(tmp,axis=0)
                vec = vec.reshape((1,dim))
                out = np.concatenate((out,vec),axis=0)
            else:
                out = np.mean(tmp,axis=0).reshape((1,dim))                                            
        else:
            pass
        
        
    return out

# Putting It All Together To Assemble Dataset

Now, putting all the preprocessing steps together we assemble our dataset...

In [10]:
import os
import numpy as np

# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

# load data in appropriate form
def load_data(path):
    data, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in os.listdir(folder):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            text = tokenize(text)
            text = stop_word_removal(text)
            text = reg_expressions(text)
            data.append(text)
            sentiments.append(sentiment)
    data_np = np.array(data)
    data, sentiments = unison_shuffle_data(data_np, sentiments)
    
    return data, sentiments

train_path = os.path.join('aclImdb', 'train')
test_path = os.path.join('aclImdb', 'test')
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


In [11]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)),size=(Nsamp*2,),replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

print("DEBUG::data_train::")
print(data_train)

DEBUG::data_train::
[list(['following', 'success', 'awful', 'gilligans', 'island', 'tv', 'movie', 'reruns', 'number', 'tv', 'movies', 'made', 's', 'reuniting', 'casts', 'classic', 'shows', 'most', 'movies', 'completely', 'missed', 'boat', 'far', 'recapturing', 'humor', 'made', 'shows', 'special', 'the', 'munsters', 'revenge', 'among', 'disappointing', 'goes', 'laurel', 'hardytype', 'comedy', 'style', 'really', 'original', 'series', 'yvonne', 'de', 'carlo', 'wonderful', 'comedienne', 'essential', 'series', 'completely', 'wasted', '', 'less', 'time', '', 'minute', 'movie', 'single', '', 'minute', 'episode', 'and', 'since', 'roles', 'eddie', 'marilyn', 'movie', 'nothing', 'cameos', 'point', 'making', 'younger', 'recasting', 'them', 'with', 'little', 'rewriting', 'could', 'used', 'butch', 'patrick', 'pat', 'priest', 'although', 'seeing', 'munsters', 'spooky', 'relatives', 'nice', 'touch', 'i', 'enjoy', 'sid', 'caesar', 'hammy', 'mad', 'scientist', 'mainly', 'theres', 'much', 'enough', 'und

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes

In [12]:
unique_elements, counts_elements = np.unique(header, return_counts=True)
print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[1011  989]


**Featurize and Create Labels**

In [13]:
EmbeddingVectors = assemble_embedding_vectors(data_train)
print(EmbeddingVectors)

[[-0.07151219 -0.05331818 -0.18072508 ...  0.12663087  0.12337326
   0.03595877]
 [-0.03273259 -0.01451753 -0.21075499 ...  0.13005976  0.13677667
   0.04476473]
 [-0.10236117 -0.01661318 -0.1793083  ...  0.06386437  0.09658311
   0.01304951]
 ...
 [-0.08675506  0.01669579 -0.12758039 ...  0.14810123  0.12053204
   0.01121135]
 [-0.07245742 -0.01820484 -0.18501921 ...  0.10941439  0.13572739
   0.05552756]
 [-0.03165541 -0.05540255 -0.16803874 ...  0.13220501  0.10361343
   0.01665439]]


In [14]:
data = EmbeddingVectors

idx = int(0.7*data.shape[0])

# 70% of data for training
train_x = data[:idx,:]
train_y = header[:idx]
# # remaining 30% for testing
test_x = data[idx:,:]
test_y = header[idx:] 

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x)
print(train_y[:5])
print(len(train_y))

train_x/train_y list details, to make sure it is of the right form:
1400
[[-0.07151219 -0.05331818 -0.18072508 ...  0.12663087  0.12337326
   0.03595877]
 [-0.03273259 -0.01451753 -0.21075499 ...  0.13005976  0.13677667
   0.04476473]
 [-0.10236117 -0.01661318 -0.1793083  ...  0.06386437  0.09658311
   0.01304951]
 ...
 [-0.06014239 -0.04210255 -0.1780156  ...  0.138171    0.07942397
   0.03878456]
 [-0.02130712 -0.02819808 -0.18746395 ...  0.17764199  0.1178336
   0.03452722]
 [-0.07624432 -0.03930511 -0.20063326 ...  0.11668558  0.11778102
   0.03071565]]
[0 1 0 0 1]
1400


# Logistic Regression Classifier

In [15]:
from sklearn.linear_model import LogisticRegression

def fit(train_x,train_y):
    model = LogisticRegression()

    try:
        model.fit(train_x, train_y)
    except:
        pass
    return model

model = fit(train_x,train_y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
predicted_labels = model.predict(test_x)
print("DEBUG::The logistic regression predicted labels are::")
print(predicted_labels)

DEBUG::The logistic regression predicted labels are::
[0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0 0 0
 0 0 1 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 1 0 1 0 1 1
 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0
 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1
 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0
 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1
 1 0 1 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1
 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0
 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1
 1 1 0 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1

In [17]:
from sklearn.metrics import accuracy_score

acc_score = accuracy_score(test_y, predicted_labels)

print("The logistic regression accuracy score is::")
print(acc_score)

The logistic regression accuracy score is::
0.7816666666666666


# Random Forests

In [18]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (spam, not spam?)
start_time = time.time()
clf.fit(train_x, train_y)
end_time = time.time()
print("Training the Random Forest Classifier took %3d seconds"%(end_time-start_time))

predicted_labels = clf.predict(test_x)
print("DEBUG::The RF predicted labels are::")
print(predicted_labels)

acc_score = accuracy_score(test_y, predicted_labels)

print("DEBUG::The RF testing accuracy score is::")
print(acc_score)

Training the Random Forest Classifier took   0 seconds
DEBUG::The RF predicted labels are::
[0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0
 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1
 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0
 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1
 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0
 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0
 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1
 1 0 1 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0
 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1
 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0
 1 1 1 0

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
from IPython.display import HTML
def create_download_link(title = "Download file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

#create_download_link(filename='GBMimportances.svg')

In [20]:
!rm -rf aclImdb
!rm aclImdb_v1.tar.gz